In [1]:


import pandas as pd
import time
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from xgboost import plot_tree
from xgboost import plot_importance
import matplotlib.pyplot as plt
import gc

path = '/home/icervera/workspace-python/kaggle/AdTracking/data/'

dtypes = {'ip'            : 'uint32',
          'app'           : 'uint16',
          'device'        : 'uint16',
          'os'            : 'uint16',
          'channel'       : 'uint16',
          'is_attributed' : 'uint8',
          'click_id'      : 'uint32'
          }

print('loading train data...')
start_time = time.time()
#train_df = pd.read_csv(path+"train.csv", nrows=10000000, dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'attributed_time','is_attributed'])
train_df = pd.read_csv(path+"train.csv", dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'attributed_time','is_attributed'])

# total observations: 184,903,891

print('[{}] Finished to load data'.format(time.time() - start_time))
train_df.shape

/home/icervera/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


loading train data...
[107.9268307685852] Finished to load data


(184903890, 8)

In [2]:
print('Extracting day and hour...')
#train_df['hour'] = pd.to_datetime(train_df.click_time).dt.hour.astype('uint8')
train_df['day']  = pd.to_datetime(train_df.click_time).dt.day.astype('uint8')

train_df['hour'] = pd.to_datetime(train_df.click_time).dt.second/3600 + pd.to_datetime(train_df.click_time).dt.minute/60 + pd.to_datetime(train_df.click_time).dt.hour


train_df['day_attributed']  = pd.to_datetime(train_df.attributed_time, errors='coerce').dt.day

train_df['hour_attributed'] = pd.to_datetime(train_df.attributed_time, errors='coerce').dt.second/3600 + pd.to_datetime(train_df.attributed_time, errors='coerce').dt.minute/60 + pd.to_datetime(train_df.attributed_time, errors='coerce').dt.hour

train_df.head()

Extracting day and hour...


,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,day_attributed,hour_attributed
0,83230,3,1,13,379,2017-11-06 14:32:21,NaN,0,6,14.539167,NaN,NaN
1,17357,3,1,19,379,2017-11-06 14:33:34,NaN,0,6,14.559444,NaN,NaN
2,35810,3,1,13,379,2017-11-06 14:34:12,NaN,0,6,14.570000,NaN,NaN
3,45745,14,1,13,478,2017-11-06 14:34:52,NaN,0,6,14.581111,NaN,NaN
4,161007,3,1,13,379,2017-11-06 14:35:08,NaN,0,6,14.585556,NaN,NaN


In [3]:
train_df = train_df.drop(columns=['click_time'])
train_df.head()
gc.collect()

62

In [ ]:
# Count the number of clicks by ip
ip_count = train_df.groupby('ip')['channel'].count().reset_index()
ip_count.columns = ['ip',  'click_byIp']
train_df = pd.merge(train_df, ip_count, on='ip', how='left', sort=False)
train_df['click_byIp'] = train_df['click_byIp'].astype('uint16')
train_df.head()

,ip,app,device,os,channel,attributed_time,is_attributed,day,hour,day_attributed,hour_attributed,click_byIp
0,83230,3,1,13,379,NaN,0,6,14.539167,NaN,NaN,25719
1,17357,3,1,19,379,NaN,0,6,14.559444,NaN,NaN,24133
2,35810,3,1,13,379,NaN,0,6,14.570000,NaN,NaN,9791
3,45745,14,1,13,478,NaN,0,6,14.581111,NaN,NaN,38077
4,161007,3,1,13,379,NaN,0,6,14.585556,NaN,NaN,1171


In [ ]:
train_df = train_df.sort_values(by=['ip','day','hour'], ascending=[True,True,True])
train_df.head()

In [ ]:
# calcul the time between this click and the last click: click_interval_before
# calcul the time between this click and the next click: click_interval_after
# need to sort the df by ip, day, hour before doing this


start_time = time.time()

train_df['hour-1'] = train_df['hour'].shift(1)
train_df['ip-1'] = train_df['ip'].shift(1)
train_df['day-1'] = train_df['day'].shift(1)


train_df['hour+1'] = train_df['hour'].shift(-1)
train_df['ip+1'] = train_df['ip'].shift(-1)
train_df['day+1'] = train_df['day'].shift(-1)

click_interval_before = []
click_interval_after = []
for index, row in train_df.iterrows():
    if (row['day']==row['day-1'] and row['ip']==row['ip-1']):
        click_interval_before.append(row['hour'] - row['hour-1'])
    else :
        # click_interval_before.append(24)
        click_interval_before.append(row['hour'])
    if (row['day']==row['day+1'] and row['ip']==row['ip+1']):
        click_interval_after.append(row['hour+1'] - row['hour'])
    else :
        # click_interval_after.append(24)
        click_interval_after.append(24 - row['hour'])
train_df['click_interval_before'] = click_interval_before
train_df['click_interval_after'] = click_interval_after

print('[{}] Finished to load data'.format(time.time() - start_time))
train_df.head()


In [ ]:
del click_interval_before
del click_interval_after
gc.collect()

In [ ]:
train_df = train_df.drop(['day-1', 'hour-1', 'ip-1' ], axis=1)
train_df = train_df.drop(['day+1', 'hour+1', 'ip+1' ], axis=1)
gc.collect()
train_df.head()

In [ ]:
# comportement global d'un IP via la distribution des clicks
# calcul the 0.9 quantile of hour_byIP

# Count the number of clicks by ip
#ip_count = train_df.groupby('ip')['hour'].quantile(0.8).reset_index()
#ip_count.columns = ['ip',  'hour_byIP_08quantile']
#train_df = pd.merge(train_df, ip_count, on='ip', how='left', sort=False)
#train_df.head()

In [ ]:
# comportement journalié d'un IP via la distribution des clicks sur la journée 
# calcul the 0.8 quantile of hour_byIP

# Count the number of clicks by ip
#ip_count = train_df.groupby(['ip','day'])['hour'].quantile(0.8).reset_index()
#ip_count.columns = ['ip','day',  'hour_byIP_byDay_08quantile']
#train_df = pd.merge(train_df, ip_count, on=['ip','day'], how='left', sort=False)
#train_df.head()

#train_df = train_df.drop(['click_interval_after_median_x', 'click_interval_after_median_y', 
#                          'click_interval_before_median_x', 'click_interval_before_median_y', ], axis=1)
#train_df = train_df.drop(['click_interval_after_mad',
#                          'click_interval_before_mad' ], axis=1)

In [ ]:
# calcul the median and mad of click_interval_after

ip_count = train_df.groupby('ip')['click_interval_after'].median().reset_index()
ip_count.columns = ['ip',  'click_interval_after_median']
train_df = pd.merge(train_df, ip_count, on='ip', how='left', sort=False)
#train_df['click_interval_after_median'] = train_df['click_interval_after_median'].astype('uint16')

if (FALSE):
    ip_count = train_df.groupby('ip')['click_interval_after'].mad().reset_index()
    ip_count.columns = ['ip',  'click_interval_after_mad']
    train_df = pd.merge(train_df, ip_count, on='ip', how='left', sort=False)

train_df.head()

In [ ]:
# calcul the median and mad of click_interval_before

ip_count = train_df.groupby('ip')['click_interval_before'].median().reset_index()
ip_count.columns = ['ip',  'click_interval_before_median']
train_df = pd.merge(train_df, ip_count, on='ip', how='left', sort=False)
#train_df['click_interval_after_median'] = train_df['click_interval_after_median'].astype('uint16')

if (FALSE):
    ip_count = train_df.groupby('ip')['click_interval_before'].mad().reset_index()
    ip_count.columns = ['ip',  'click_interval_before_mad']
    train_df = pd.merge(train_df, ip_count, on='ip', how='left', sort=False)
    
train_df.head()

In [ ]:
#start_time = time.time()
#click_interval_before_distance = []
#click_interval_after_distance = []
#for index, row in train_df.iterrows():
#    if (row['click_interval_before']<row['click_interval_before_median']):
#        click_interval_before_distance.append(row['click_interval_before_median']-
#                                              row['click_interval_before_mad']-
#                                              row['click_interval_before'])
#    else :
#        click_interval_before_distance.append(row['click_interval_before']-(
#                                              row['click_interval_before_median']+
#                                              row['click_interval_before_mad']))
#        
#    if (row['click_interval_after']<row['click_interval_after_median']):
#        click_interval_after_distance.append(row['click_interval_after_median']-
#                                              row['click_interval_after_mad']-
#                                              row['click_interval_after'])
#    else :
#        click_interval_after_distance.append(row['click_interval_after']-(
#                                              row['click_interval_after_median']+
#                                              row['click_interval_after_mad']))
#train_df['click_interval_before_distance'] = click_interval_before_distance
#train_df['click_interval_after_distance'] = click_interval_after_distance#
#print('[{}] Finished to load data'.format(time.time() - start_time))


train_df['click_interval_before_distance'] =np.arcsinh( train_df['click_interval_before']-train_df['click_interval_before_median'])
train_df['click_interval_after_distance'] = np.arcsinh( train_df['click_interval_after']-train_df['click_interval_after_median'] )

train_df['click_interval_before_after_dist'] = train_df['click_interval_before']+train_df['click_interval_after']


In [ ]:
train_df['download_interval'] = (train_df['day_attributed']-train_df['day'])*24 +\
    train_df['hour_attributed']-train_df['hour']
train_df.head()
gc.collect()

In [ ]:
from ggplot import *

if (FALSE) : 
    tpm_df = train_df.loc[train_df['is_attributed']==1]
    tpm_df.assign(is_attributed = lambda x: pd.Series(np.repeat(1, x.shape[0]), index=x.index)) 
    tpm2_df = train_df.loc[train_df['is_attributed']!=1].sample(18000, axis=0)
    tpm2_df.assign(is_attributed = lambda x: pd.Series(np.repeat(0, x.shape[0]), index=x.index)) 
    rdm_df = pd.concat([tpm_df,tpm2_df])

    rdm_df = rdm_df.assign(is_attributed_char = lambda x: x.is_attributed.astype(str)) 

    ggplot(rdm_df, aes('click_interval_before', fill='is_attributed_char'))+\
    geom_histogram(binwidth=0.2)

In [ ]:
if (FALSE) : 
    ska_df = rdm_df.loc[rdm_df['is_attributed']==1]
    ska_df['click_interval_after_log'] = np.log(ska_df['click_interval_after']+1)
    ska_df['download_interval_log'] = np.log(ska_df['download_interval']+1)
    ska_df['click_byChannel_log'] = np.log(ska_df['click_byChannel']+1)
    ska_df['click_byIp_log'] = np.log(ska_df['click_byIp']+1)
    ggplot(ska_df, aes('click_byIp_log', 'download_interval'))+geom_point(alpha=0.5)

In [ ]:
# NbrClickOnThisAppBefore_byDay 
# ratioClickOnThisAppBefore_byDay = NbrClickOnThisAppBefore_byDay / NbrClickOnAppsBefore_byDay

max_clicks_by_ip = train_df['click_byIp'].max()


#sample_train_df = train_df.sample(100000, axis=0)
#sample_train_df = sample_train_df.sort_values(by=['ip','day','hour'], ascending=[True,True,True])


start_time = time.time()


NbrClickOnThisAppBefore_byDay = []
NbrClickBefore_byDay = []
NbrClickOnThisAppAfter_byDay = []
#NbrClick_byDay = []
for index, row in train_df.iterrows():
    # create a tpm_df which have all the row with the same ip and same day before the index ip
    if(index<(max_clicks_by_ip+1)):
        index_min=0
    else:
        index_min=index-max_clicks_by_ip
    if(index>0):
        tpm_df = train_df.iloc[(index_min):(index)]
        #tpm_df = train_df.iloc[0:(index)]
        tpm_df = tpm_df.loc[(tpm_df['ip'] == row['ip']) & (tpm_df['day'] == row['day']) ]
        clickOnThisAppBefore = np.sum(tpm_df['app']==row['app'])
        click = tpm_df.shape[0]
        #ip_count = tpm_df.groupby(['ip','day','app'])['device'].count().reset_index()
        #ip_count.columns = ['ip',  'day','app','value']
        #clickOnApp = ip_count.loc[(ip_count['ip'] == row['ip']) & (ip_count['day'] == row['day']) & (ip_count['app'] == row['app'])   ]['value'].iloc[0]
        #click = ip_count.loc[(ip_count['ip'] == row['ip']) & (ip_count['day'] == row['day'])].shape[0]
    else:
        clickOnThisAppBefore=0
        click=0
        
    tpm_dfAfter = train_df.iloc[(index+1):(index+index_min)]
    #tpm_df = train_df.iloc[0:(index)]
    tpm_dfAfter = tpm_dfAfter.loc[(tpm_dfAfter['ip'] == row['ip']) & (tpm_dfAfter['day'] == row['day']) ]
    clickOnThisAppAfter = np.sum(tpm_dfAfter['app']==row['app'])
        
    #clickTotal = sample_train_df.loc[(sample_train_df['ip'] == row['ip']) & (sample_train_df['day'] == row['day']) ].shape[0]
    NbrClickOnThisAppBefore_byDay.append(clickOnThisAppBefore)
    NbrClickBefore_byDay.append(click)
    NbrClickOnThisAppAfter_byDay.append(clickOnThisAppAfter)
    #NbrClick_byDay.append(clickTotal)
train_df['click_onAppBefore_byDayByIp'] = NbrClickOnThisAppBefore_byDay
train_df['click_Before_byDayByIp'] = NbrClickBefore_byDay
train_df['click_onAppAfter_byDayByIp'] = NbrClickOnThisAppAfter_byDay

# le 'NbrClick_byDay' on peut le faire plus rapidement avec un groupBy sans la boucle for
# sample_train_df['NbrClick_byDay'] = NbrClick_byDay 

print('[{}] Finished to load data'.format(time.time() - start_time))

In [ ]:
del NbrClickOnThisAppBefore_byDay
del NbrClickBefore_byDay
del NbrClickOnThisAppAfter_byDay
del tpm_df
del tpm_dfAfter
gc.collect()

In [ ]:
# Count the number of clicks by day by ip
ip_count = train_df.groupby(['ip','day'])['channel'].count().reset_index()
ip_count.columns = ['ip','day',  'click_byDayByIp']
train_df = pd.merge(train_df, ip_count, on=['ip','day'], how='left', sort=False)
train_df['click_byDayByIp'] = train_df['click_byDayByIp'].astype('uint16')
train_df.head()

In [ ]:
# est ce que cela peut etre bien de faire de la regression sur le temps que lutilisateur met pr telecharger? 
# au lieu de faire de la classification?
train_df['ratio_click_onAppBefore_byDayByIp'] = (train_df['click_onAppBefore_byDayByIp'])/(train_df['click_byDayByIp'])
train_df['ratio_click_onAppAfter_byDayByIp'] = (train_df['click_onAppAfter_byDayByIp'])/(train_df['click_byDayByIp'])
train_df.head()

In [ ]:
# trouver qlqchose de mieux que click interval dist
#from ggplot import *

train_df['click_interval_log'] = np.log(train_df['click_interval']+1)
#train_df['click_interval_dist_log'] = np.log(train_df['click_interval_dist']+1)


In [ ]:
# train_df['ClickOnThisAppBefore_byDay'] = train_df['click_onAppBefore_byDayByIp']>0
# print (train_df[['ClickOnThisAppBefore_byDay', 'is_attributed']].groupby(['ClickOnThisAppBefore_byDay'], as_index=False).mean())

In [ ]:
train_df['click_byDayByIp_log'] = np.log(train_df['click_byDayByIp'])


In [ ]:
gc.collect()

In [ ]:
# Count the number of app_byChannel
channel_uniqueApp = train_df.groupby(['ip'])['app'].nunique().reset_index()
channel_uniqueApp.columns = ['ip','app_byIp']
train_df = pd.merge(train_df, channel_uniqueApp, on=['ip'], how='left', sort=False)
train_df.head()

In [ ]:
# Count the number of app_byChannel
channel_uniqueApp = train_df.groupby(['channel'])['app'].nunique().reset_index()
channel_uniqueApp.columns = ['channel','app_byChannel']
train_df = pd.merge(train_df, channel_uniqueApp, on=['channel'], how='left', sort=False)
train_df.head()



In [ ]:
# count the number of click by channel
channel_count = train_df.groupby(['channel'])['app'].count().reset_index()
channel_count.columns = ['channel','click_byChannel']
train_df = pd.merge(train_df, channel_count, on=['channel'], how='left', sort=False)
train_df.head()



In [ ]:
# count the number of channel by ip
channel_by_ip = train_df.groupby(['ip'])['channel'].nunique().reset_index()
channel_by_ip.columns = ['ip','channel_byIP']
train_df = pd.merge(train_df, channel_by_ip, on=['ip'], how='left', sort=False)
train_df.head()

In [ ]:
train_df['click_byIp_log'] = np.log(train_df['click_byIp'])
train_df['channel_byIP_log'] = np.log(train_df['channel_byIP'])


In [ ]:
gc.collect()

In [ ]:
# ratio for channel
train_df['ratio_byChannel'] = (train_df['click_byChannel']/train_df['app_byChannel'])
train_df.head()

In [ ]:
gc.collect()

In [ ]:
train_df.to_csv(path+"trainAll_newFeatures.csv")